In [1]:
import mpmath as mp

from code.python.mpmath_integration import quad_phi

In [2]:
x = mp.mpf('3')
alpha = mp.mpf('5')
beta = mp.mpf('1/2')
mu = mp.mpf('1')
delta = mp.mpf('4')

xmu = x - mu

gamma = mp.sqrt(alpha ** 2 - beta ** 2)
omega = mp.sqrt(xmu ** 2 + delta ** 2)

### 1. Expansion b -> 0

In [4]:
a = x-mu
b = -beta

In [5]:
def normcdf(x):
    return mp.erfc(-x / mp.sqrt(mp.mpf('2'))) / mp.mpf('2')

In [6]:
t = 1

normcdf(a / mp.sqrt(t) + b * mp.sqrt(t))

mpf('0.93319279873114191')

In [9]:
A = mp.exp(-a**2/2 * t) / mp.sqrt(2*mp.pi) * mp.sqrt(t)

K = 40
S = 0
for k in range(K):
    S += (-1) ** k * b ** (k + 1) / mp.factorial(k + 1) * mp.hermite(k, a / mp.sqrt(2*t)) * (t / 2) ** (k / 2)

normcdf(a / mp.sqrt(t)) + A * S

mpf('0.93319279873114191')

### 2. Expansion

In [10]:
mp_result = quad_phi(x, alpha, beta, mu, delta, digits=100)
mp_result

mpf('0.95997754054946574')

In [11]:
quad_phi(x, gamma, 0, mu, delta, digits=100)

mpf('0.98630189280527029')

In [12]:
C = delta * mp.exp(delta * gamma) / (2 * mp.pi)

In [13]:
mp.mp.dps = 100

N = 20
s = 0
for k in range(N):
    r = (-1) ** k * (-beta) ** (k + 1) / mp.factorial(k + 1) / 2 ** (k/2)
    q = mp.quad(lambda t: t ** (k/2 - 1) * mp.hermite(k, xmu / mp.sqrt(2 * t)) * mp.exp(-omega ** 2 / 2 / t - gamma ** 2 / 2 * t), [0, mp.inf])
    s += r * q

C * s

mpf('-0.02632435225580452909447606912714988231711442630495450207934465163741921938323271904959628967233365243257')

In [14]:
r1 = quad_phi(x, gamma, 0, mu, delta, digits=100) + C * s

In [16]:
float(abs(r1 / mp_result - 1))

2.8102413509359047e-17

In [41]:
j = 4
mp.quad(lambda t: t ** (j - 1) * mp.exp(-omega ** 2 / 2 / t - gamma ** 2 / 2 * t), [0, mp.inf])

mpf('0.0000000001066502480955874235763167453152610739522494459182923638751868470308621807431170453416998408301908371817')

In [44]:
2 * (omega / gamma) ** j * mp.besselk(j, gamma * omega)

mpf('0.0000000001066502480955874235763167453152610739522494459182923638751868470308621807431170453416998408301908371826')

In [50]:
k = 5
mp.quad(lambda t: t ** (k/2 - 1) * mp.hermite(k, xmu / mp.sqrt(2 * t)) * mp.exp(-omega ** 2 / 2 / t - gamma ** 2 / 2 * t), [0, mp.inf])

mpf('-0.000000009784244885156514763525857922935277896236834477751819148067887126750944833870598658603585196859960924589')

In [51]:
s = 0
for j in range(int(mp.floor(k/2)) + 1):
    s += (-1) ** j / mp.factorial(j) / mp.factorial(k - 2 * j) * mp.besselk(j, gamma * omega) * (omega / gamma /2 / xmu ** 2) ** j

2 * mp.factorial(k) * s * (mp.sqrt(2) * xmu) ** k

mpf('-0.000000009784244885156514763525857922935277896236834477751819148067887126750944833870598658603585196859960924483')

#### Final expansion

In [52]:
N = 20

In [53]:
def Ak(k):
    s = 0
    for j in range(int(mp.floor(k/2)) + 1):
        s += (-1) ** j / mp.factorial(j) / mp.factorial(k - 2 * j) * (omega / gamma /2 / xmu ** 2) ** j * mp.besselk(j, gamma * omega)
    return s

In [54]:
s = 0
for k in range(N):
    s += (beta * (x-mu)) ** k / (k + 1) * Ak(k)

In [58]:
r2 = quad_phi(x, gamma, 0, mu, delta, digits=100) - beta * delta * mp.exp(delta * gamma) / mp.pi * s

In [59]:
float(abs(r2 / mp_result - 1))

2.9168251091814864e-17